# Project - ECG Arrhythmia Classification
### Natalia Kolińska, Alicja Smaruj, Dorota Woźna, Kacper Zielak


#### Imports

In [ ]:
#pip install -r requirements.txt

In [ ]:
#Załadowanie pakietów
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler
from sklearn import tree
from sklearn.model_selection import StratifiedKFold
from sklearn.dummy import DummyClassifier
import numpy as np
from scikit_posthocs import posthoc_dunn
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, balanced_accuracy_score
import missingno as msno
from sklearn.decomposition import PCA
from sklearn.base import BaseEstimator
from sklearn.base import TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix

#### Load data
Source: [Kaggle](https://www.kaggle.com/datasets/sadmansakib7/ecg-arrhythmia-classification-dataset?select=Sudden+Cardiac+Death+Holter+Database.csv)

In [ ]:
df_1 = pd.read_csv("data/INCART 2-lead Arrhythmia Database.csv")
df_2 = pd.read_csv("data/MIT-BIH Arrhythmia Database.csv")
df_3 = pd.read_csv("data/MIT-BIH Supraventricular Arrhythmia Database.csv")
test_data = pd.read_csv("data/Sudden Cardiac Death Holter Database.csv")

In [ ]:
train_data = pd.concat([df_1, df_2, df_3])
train_data.head()

In [ ]:
test_data.head()

#### Explore data

Dependent variable
- N (Normal),
- SVEB (Supraventricular ectopic beat),
- VEB (Ventricular ectopic beat),
- F (Fusion beat),
- Q (Unknown beat).

In [ ]:
train_data.groupby("type").size()

In [ ]:
test_data.groupby("type").size()

Zbiór danych zawiera cechy utworzone na podstawie wynikow EKG serca. Podawane są wysokości lub szerokości wyliczone między specyficznymi momentami w szeregu czasowym. Pozwala to na zastosowanie uczenia maszynowego do klasyfikacji szeregów czasowych.

<img src="images/wykres_ecg.png" alt="Wykres" width="400"/>
<img src="images/prepost.png" alt="Wykres2" width="400"/>

Autorzy zbioru utworzyli 16 zmiennych. Zbadali je na surowych szeregach czasowych (a) oraz przetworzonych szeregach na których usunięto szum (b).Stąd 32 kolumny z pomiarami. Surowe dane mają przedrostek '0' a przetworzone '1'.

<img src="images/LeadAB.png" alt="Wykres2" width="800"/>

Check if anything is null

In [ ]:
train_data.isnull().any()

In [ ]:
test_data.isnull().any()

Check how many percentage are null values in test_data

In [ ]:
round(test_data.isnull().sum() * 100 / len(test_data), 2)

In [ ]:
msno.matrix(test_data)

Check if outliers influence arrhythmia

In [ ]:
print(
    "Q1:",
    train_data["0_pPeak"].quantile(0.25),
    "Q2:",
    train_data["0_pPeak"].quantile(0.5),
    "Q3:",
    train_data["0_pPeak"].quantile(0.75),
)
print("Min:", min(train_data["0_pPeak"]), "Max:", max(train_data["0_pPeak"]))

In [ ]:
q3 = train_data["0_pPeak"].quantile(0.75)
print("Amount of outliers:", len(train_data.loc[train_data["0_pPeak"] > q3]))
print("Percentage of type in the outlier group")
train_data.loc[train_data["0_pPeak"] > q3].groupby("type").size() * 100 / len(train_data.loc[train_data["0_pPeak"] > q3])

In [ ]:
#Jest bardzo dużo outlierów, ale czy powinniśmy je usunąć lub zastąpić? Możliwe, że to były mocniejsze uderzenia serca u zdrowych osób.

#### Prepare data

Remove 'record' column, because the name of the subject/patient is irrelevant in this study.

In [ ]:
train_data = train_data.drop("record", axis=1)
test_data = test_data.drop("record", axis=1)

Remove null observations from test_data

In [ ]:
test_data = test_data.dropna()
# len(test_data)

Split X and y

In [ ]:
y_train = train_data["type"]
X_train = train_data.drop(columns=["type"])

In [ ]:
y_test = test_data["type"]
X_test = test_data.drop(columns=["type"])

#### Visualisation

In [ ]:
corr_matrix = X_train.corr()
plt.figure(figsize=(9,7))
sns.set_theme("notebook")
sns.set_palette("PuBuGn_d")
sns.heatmap(corr_matrix, cmap="coolwarm", annot=False)
plt.savefig("images/corplot.png")

In [ ]:
train_data0 = train_data[["type", *[column_name for column_name in train_data.columns if column_name.startswith("0_")]]]

In [ ]:
train_data1 = train_data[["type", *[column_name for column_name in train_data.columns if column_name.startswith("1_")]]]

Wykresy pudełkowe dla danych surowych (0) i przefiltrowanych (1) podzielone na grupy:
* pre-RR i post-RR
* peaks
* intervals
* morphology

In [ ]:
# Zbudowanie wykresów pudełkowych trwa ok. 3 minuty.
# Jeśli nie potrzebujesz wersji interaktywnej, możesz skorztstać z zapisanych obrazów. 

prepost_0=["0_pre-RR", "0_post-RR"]
data_to_box_0 = pd.melt(train_data0[["type", *prepost_0]], id_vars="type", value_vars=prepost_0)
fig1 = px.box(data_to_box_0, y="value", x="variable", color="type", width=1000)
fig1.write_image("images/box_RR_0.png")

prepost_1=["1_pre-RR", "1_post-RR"]
data_to_box_1 = pd.melt(train_data1[["type", *prepost_1]], id_vars="type", value_vars=prepost_1)
fig2 = px.box(data_to_box_1, y="value", x="variable", color="type", width=1000)
fig2.write_image("images/box_RR_1.png")

peaks_0=["0_pPeak", "0_tPeak", "0_rPeak", "0_sPeak", "0_qPeak"]
data_to_box_3 = pd.melt(train_data0[["type",*peaks_0]], id_vars="type", value_vars=peaks_0)
fig3= px.box(data_to_box_3, y="value", x="variable", color="type", width=1000)
fig3.write_image("images/box_peaks_0.png")

peaks_1=["1_pPeak", "1_tPeak", "1_rPeak", "1_sPeak", "1_qPeak"]
data_to_box4 = pd.melt(train_data1[["type", *peaks_1]], id_vars="type", value_vars=peaks_1)
fig4= px.box(data_to_box4, y="value", x="variable", color="type", width=1000)
fig4.write_image("images/box_peaks_1.png")

intervals_0=["0_qrs_interval", "0_pq_interval", "0_qt_interval", "0_st_interval"]
data_to_box5 = pd.melt(train_data0[["type", *intervals_0]], id_vars="type", value_vars=intervals_0)
fig5 = px.box(data_to_box5, y="value", x="variable", color="type", width=1000)
fig5.write_image("images/box_intervals_0.png")

intervals_1=["1_qrs_interval", "1_pq_interval", "1_qt_interval", "1_st_interval"]
data_to_box6 = pd.melt(train_data1[["type", *intervals_1]], id_vars="type", value_vars=intervals_1)
fig6 = px.box(data_to_box6, y="value", x="variable", color="type", width=1000)
fig6.write_image("images/box_intervals_1.png")

morphs_0=["0_qrs_morph0", "0_qrs_morph1", "0_qrs_morph2", "0_qrs_morph3", "0_qrs_morph4",]
data_to_box7 = pd.melt(train_data0[["type", *morphs_0]], id_vars="type", value_vars=morphs_0)
fig7 = px.box(data_to_box7, y="value", x="variable", color="type")
fig7.write_image("images/box_morphs_0.png")

morphs_1=["1_qrs_morph0", "1_qrs_morph1", "1_qrs_morph2", "1_qrs_morph3", "1_qrs_morph4",]
data_to_box8 = pd.melt(train_data1[["type", *morphs_1]], id_vars="type", value_vars=morphs_1)
fig8 = px.box(data_to_box8, y="value", x="variable", color="type")
fig8.write_image("images/box_morphs_1.png")

#### Standaryzacja oryginalnych danych

In [ ]:
std_all = StandardScaler()
train_data_std = std_all.fit_transform(X_train)
train_data_std = pd.DataFrame(train_data_std, columns=X_train.columns)

test_data_std = std_all.transform(X_test)
test_data_std = pd.DataFrame(test_data_std, columns=X_test.columns)

#### PCA

In [ ]:
pca = PCA(n_components=2)
components = pca.fit_transform(train_data_std)

fig = px.scatter(components, x=0, y=1, color=y_train, width=800)
fig.write_image("images/pca_2d.png")
# fig.show()

#### Dummy model

Stworzono "głupi" model, który będzie służył do porównań, czy nasze modele uzyskują lepsze wyniki. Przyjęto strategię most-frequent, co będzie skutkowało zaklysyfikowaniem wszystkich obserwacji jako N (normalne).

In [ ]:
dummy_model = DummyClassifier(strategy="most_frequent")
dummy_model.fit(train_data_std, y_train)
y_pred = dummy_model.predict(test_data_std)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

balanced_accuracy = balanced_accuracy_score(y_test, y_pred)
print("Balanced accuracy:", balanced_accuracy)

precision = precision_score(y_test, y_pred, average="weighted", zero_division=np.nan)
print("Precision:", precision)

recall = recall_score(y_test, y_pred, average="weighted")
print("Sensitivity (recall):", recall)

# specificity = specificity_score(y_test, y_pred)
# print("Specificity :", specificity, average="weighted")

f1 = f1_score(y_test, y_pred, average="weighted")
print("F1-Score:", f1)

#### Model 1 - Natalia

#### Model 2 - Dorota

##### Krok 1: Stworzenie nowych zmiennych i zbadanie istotności ich wpływu na rozróżnienie grup

In [ ]:
# Na podstawie wykresów pudełkowych box_RR_0 i box_RR_1 można zauważyć, że w przypadku zaburzenia SVEB, preRR jest często krótsze niż postRR,
# dlatego dodano dwie zmienne opisujące ich różnice.
data_new=pd.DataFrame()
data_new["0_RR_diff"] = X_train["0_pre-RR"] - X_train["0_post-RR"]
data_new["1_RR_diff"] = X_train["1_pre-RR"] - X_train["1_post-RR"]
data_new["type"] =  y_train

def perform_dunn_test(data, column, group="type", p_adjust="fdr_bh"):
    dunn_df = posthoc_dunn(data, val_col=column, group_col=group, p_adjust=p_adjust)
    remove = np.tril(np.ones(dunn_df.shape), k=0).astype(bool)
    dunn_df[remove] = np.nan
    molten_df = dunn_df.melt(ignore_index=False).reset_index().dropna()
    molten_df.rename(columns={"value": f"{column}_sign"}, inplace=True)
    return molten_df

molten_df1 = perform_dunn_test(data_new, "0_RR_diff")
molten_df2 = perform_dunn_test(data_new, "1_RR_diff")

dunn_summ = pd.merge(molten_df1, molten_df2)

# wskazanie istotnych
def color_red(val):
    color = "red" if val <= 0.05 else "black"
    return f"color: {color}"

# Stworzenie obiektu Styler i zastosowanie funkcji do kolumny "0_RR_diff"
dunn_summ = dunn_summ.style.applymap(color_red, subset=["0_RR_diff_sign",	"1_RR_diff_sign"])
dunn_summ

Powyższe wyniki potwierdzają, że nowo utworzone zmienne istotnie różnicują wszystkie pary typów poza F i N. Następnie przeprowadzimy standaryzację dla nowych zmiennych.

##### Krok 2: pre-processing danych treningowych i testowych z nowymi zmiennymi

In [ ]:
X_train["0_RR_diff"] = X_train["0_pre-RR"] - X_train["0_post-RR"]
X_train["1_RR_diff"] = X_train["1_pre-RR"] - X_train["1_post-RR"]

X_test["0_RR_diff"] = X_test["0_pre-RR"] - X_test["0_post-RR"]
X_test["1_RR_diff"] = X_test["1_pre-RR"] - X_test["1_post-RR"]

In [ ]:
pipeline= Pipeline(
    [
        ('std_scaler', StandardScaler())
    ]
)

In [ ]:
train_data_prep = pipeline.fit_transform(X_train)
test_data_prep = pipeline.transform(X_test)

##### Krok 3: Budowa drzew decyzyjnych

In [ ]:
# drzewo decyzyjne z walidacją krzyżową i zbalansowaniem klas przy podziale na treningowe i testowe

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=10)

ma_train = []
ma_test = []
for j in range(1, 21):
    model = tree.DecisionTreeClassifier(random_state=10, max_depth=j)
    a_test = []
    a_tren = []

    for i, (train_index, test_index) in enumerate(skf.split(train_data_prep, y_train)):
        model.fit(X_train.iloc[train_index], y_train.iloc[train_index])
        a_test.append(model.score(X_train.iloc[test_index], y_train.iloc[test_index]).round(4))
        a_tren.append(model.score(X_train.iloc[train_index], y_train.iloc[train_index]).round(4))

    ma_test.append(np.mean(a_test))
    ma_train.append(np.mean(a_tren))

##### Wybór głębokości drzewa

In [ ]:
depths = np.arange(1, 21)
plt.plot(depths, ma_test, label="Train Accuracy", color="blue")
plt.plot(depths, ma_train, label="Test Accuracy", color="red")
plt.xlabel("Max Depth")
plt.xticks(range(21))
plt.ylabel("Accuracy Score")
plt.title("Grid Search Results for Decision Tree")
plt.legend()
plt.grid(True)
plt.savefig("images/grid_search_trees.png")
plt.show()

Wybrano głębokość drzewa równą 8.

##### Zbudowanie modelu na wszystkich danych treningowych

In [ ]:
model = tree.DecisionTreeClassifier(random_state=10, max_depth=8)
model.fit(train_data_prep, y_train)

##### Ocena dokładności

In [ ]:
y_pred = model.predict(test_data_prep)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


balanced_accuracy = balanced_accuracy_score(y_test, y_pred)
print("Balanced accuracy:", balanced_accuracy)

precision = precision_score(y_test, y_pred, average="weighted")
print("Precision:", precision)

recall = recall_score(y_test, y_pred, average="weighted")
print("Sensivity (recall):", recall)

f1 = f1_score(y_test, y_pred, average="weighted")
print("F1-Score:", f1)

In [ ]:
types=np.unique(y_test.values)
conf_matrix=pd.DataFrame(confusion_matrix(y_test, y_pred, labels=types), columns=types, index=types)
percentage=conf_matrix.div(conf_matrix.sum(axis=1), axis=0)
percentage = percentage.fillna(0)
plt.figure(figsize=(6, 4))
sns.heatmap(percentage, fmt='.2%', annot=True, annot_kws={"size": 12})
plt.xlabel("Predicted label");
plt.ylabel("True label");

Obserwacje typu F były w 92,86% przypadków mylone jako N.
99,39% wartości normalnych jest prawidłowo zaklysyfikowanych.
Żadna obserwacja typu Q nie została zaklasyfikowana poprawnie. Często były mylone z N lub SVEB.
Obserwacje typu SVEB były w 97,01% przypadków mylone z N.

##### Zmienne istotne

In [ ]:
indeksy = np.where(model.feature_importances_!=0)[0]
variables= [X_train.columns[i] for i in indeksy]
importances = model.feature_importances_[indeksy]

# sortowanie
importances, variables= zip(*sorted(zip(importances, variables), reverse=False))

# Tworzenie wykresu słupkowego z niezerowymi wartościami
fig, ax = plt.subplots(figsize=(7, 5))
plt.barh(variables[:15], importances[:15])

plt.xlabel("Importance")
plt.ylabel("Feature")
plt.show()

#### Model 3 - Kacper